# Static Huffman Algorithm

In [94]:
import heapq 
import bitarray as bit

class Node:
    def __init__(self , val , char = None , left = None , right = None):
        self.val = val
        self.char = char
        self.left = left
        self.right = right
    
    def __lt__(self,other):
        return self.val < other.val


class Static_Huffman:
    def __init__(self,text):
        self.text = text
        self.codes = dict()
        self.freq = self.prepare_freq(text)
        self.root = None
        self.create_tree()

    def prepare_freq(self,text):

        out = dict()

        for letter in text:
            out[letter] = out.get(letter,0) + 1

        return out


    def create_tree(self):
        
        Nodes = []

        for char in self.freq:
            heapq.heappush(Nodes , Node(self.freq[char] , char))
        
        while len(Nodes) > 1:
            l = heapq.heappop(Nodes)
            r = heapq.heappop(Nodes)

            heapq.heappush(Nodes, Node(l.val + r.val , left=l , right= r))
        
        self.root = Nodes[0]

        self.create_codes(self.root , self.codes ,bit.bitarray())

    def create_codes(self, node, arrays, array):
        if node.char is not None:
            arrays[node.char] = array

        array_cpy = array.copy()
        if node.left is not None:
            array.append(0)
            self.create_codes(node.left, arrays, array)

        if node.right is not None:
            array = array_cpy
            array.append(1)
            self.create_codes(node.right, arrays, array)
    
    def encode_text(self):

        output = bit.bitarray()

        for char in self.text:
            output.extend(self.codes[char])
        
        return output
    
    def decode_text(self , code):
        node = self.root
        output = ""

        for bit in code:
            if bit == 0:
                node = node.left
            else:
                node = node.right

            if node.left == None and node.right == None:
                output += node.char
                node = self.root
        
        return output


# Dynamic Huffman Algorithm


In [95]:
class Node2:
    def __init__(self, char, parent=None, left=None, right=None, weight=0,):
        self.parent = parent
        self.left = left
        self.right = right
        self.weight = weight
        self.char = char


class Dynamic_Huffman:

    def __init__(self):
    
        self.revealed = [None]*10000
        self.nodes = []
        self.NYT = Node2("NYT")
        self.root = self.NYT
    
    def get_string_code(self, char , node , current_code=''):
        if not node.left and not node.right:
            if node.char == char:
                return current_code
            else:
                return ''
        else:

            tmp =''            

            if node.left:
                tmp = self.get_string_code(char , node.left , current_code + '0')
            if node.right and not tmp:
                tmp = self.get_string_code(char , node.right , current_code +'1')
            

            return tmp
    
    def get_largest(self , weight):
        for node in reversed(self.nodes):
            if node.weight == weight:
                return node
    
    def swap(self , first , second):

        idf , ids = self.nodes.index(first), self.nodes.index(second)
        self.nodes[idf], self.nodes[ids] = self.nodes[ids], self.nodes[idf]
        tmp = first.parent
        first.parent = second.parent
        second.parent = tmp

        if first.parent.left == second:
            first.parent.left = first
        else:
            first.parent.right = first

        if second.parent.left == first:
            second.parent.left = second
        else:
            second.parent.right = second


    def insert(self, char):

        node = self.revealed[ord(char)]

        if not node:
            new = Node2(char , weight=1)
            new_parent =Node2('' , self.NYT.parent , self.NYT , right = new , weight=1) 
            new.parent = new_parent
            self.NYT.parent = new_parent

            if new_parent.parent:
                new_parent.parent.left = new_parent
            else:
                self.root = new_parent

            self.nodes.insert(0, new_parent)
            self.nodes.insert(0, new)
            self.revealed[ord(char)] = new

            node = new_parent.parent
        
        while node:

            largest = self.get_largest(node.weight)

            if(node != largest and node.parent != largest and node != largest.parent):
                self.swap(node,largest)
            
            node.weight +=1
            node = node.parent
    
    def encode_text(self, text):

        output = bit.bitarray()

        temp =''

        for char in text:
           
            if self.revealed[ord(char)]:
               temp+=self.get_string_code(char , self.root)
            else:
                temp+=self.get_string_code('NYT', self.root)
                
            self.insert(char)
        
        for b in temp:
            output.append(int(b))

        return output , temp
    
    def decode_text(self , code):

     
        output = ''

        char = chr(int(code[0:8] , 2))
        
        output+=char

        self.insert(char)

        node = self.root

        i = 8 

        while i < len(code):

            if int(code[i]) == 0:
                node = node.left
            else:
                node = node.right

            if node.char:

                if node.char == "NYT":
                    char = chr(int(code[i : i+8] , 2))
                    i+=8
                
                output += char
                self.insert(char)
                node = self.root
            
            i+=1
        
        return output


            

            



        
        
        
        




                
            

        





In [103]:
import os , time


def compression_test(file):
   with open("./" + current_dir + "/" + file, encoding="utf8") as f:
      text = f.read()

   static_tree = Static_Huffman(text)
   static_tree.create_tree()
   encode_s = time.time()
   static_code = static_tree.encode_text()
   encode_end = time.time()

   decode_s = time.time()
   decoded = static_tree.decode_text(static_code)
   decode_end = time.time()

   coded_file = file[:-4] + "_test.txt"

   with open(coded_file, "wb+") as f:
      static_code.tofile(f)

   result = (1 - os.path.getsize(coded_file) / os.path.getsize("./" + current_dir  + "/" + file)) * 100

   print("Static Huffman compression ratio for " + file + " is : " , result ,"%")

   dynamic_tree = Dynamic_Huffman()
   encode2_s = time.time()
   dynamic_code , s = dynamic_tree.encode_text(text)
   encode2_end = time.time()

   decode2_s = time.time()
   decoded = dynamic_tree.decode_text(s)
   decode2_end = time.time()

   coded_file2 = file[:-4] + "2_test.txt"

   with open(coded_file2, "wb+") as f:
      dynamic_code.tofile(f)

   result = (1 - os.path.getsize(coded_file2) / os.path.getsize("./" + current_dir  + "/" + file)) * 100

   print("Dynamic Huffman compression ratio for " + file + " is : " , result ,"%")

   print("Average Static Encode Time : " , encode_end-encode_s)
   print("Average Static Decode Time: " , decode_end - decode_s)
   print("Dynamic Encode Time : " , encode2_end-encode2_s)
   print("Dynamic Decode Time: " , decode2_end - decode2_s)


   






In [104]:
def make_tests():

    dirs = ["Gutenberg" , "Linux" , "Random"]
    global current_dir

    for dir in dirs:

        current_dir = dir 

        print("Running tests for " + dir + " files")
        print("")

        for file in os.listdir(dir):
            print(file)
            compression_test(file)
        
        print("-----------------")
        print("")
            

In [105]:
make_tests()

Running tests for Gutenberg files

100_kb_file_gutenberg.txt
Static Huffman compression ratio for 100_kb_file_gutenberg.txt is :  45.91437801340693 %
Dynamic Huffman compression ratio for 100_kb_file_gutenberg.txt is :  45.88042413247834 %
Average Static Encode Time :  0.04099869728088379
Average Static Decode Time:  0.22228240966796875
Dynamic Encode Time :  12.372391700744629
Dynamic Decode Time:  1.4277031421661377
10_kb_file_gutenberg.txt
Static Huffman compression ratio for 10_kb_file_gutenberg.txt is :  45.20392749244713 %
Dynamic Huffman compression ratio for 10_kb_file_gutenberg.txt is :  44.99622356495468 %
Average Static Encode Time :  0.0030558109283447266
Average Static Decode Time:  0.02195596694946289
Dynamic Encode Time :  1.3490798473358154
Dynamic Decode Time:  0.15381407737731934
1_kb_file_gutenberg.txt
Static Huffman compression ratio for 1_kb_file_gutenberg.txt is :  45.19317160826595 %
Dynamic Huffman compression ratio for 1_kb_file_gutenberg.txt is :  44.474393530